<a href="https://colab.research.google.com/github/seznam/IT-akademie-bigdata/blob/main/big-data/notebooks/001_introduction_to_apache_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is Apache Spark & why do we need it



Imagine you were employed by a real estate agency to help them raise the income by predicting how long will it take from making a contract with the seller until a buyer is found based on futures like price, location, estate type, etc.

To accomplish this goal, you could split this task into several parts:
- gather enough data from various sources
- transform to a common standardized format
- extract key attributes for the prediction
- apply machine learning algorithms

In this notebook, we are going to focus on the first part of the problem, which is:
- Extract
- Transform
- Load

Or ETL, for short.

We picked Apache Spark for this task as it is a distributed data processing framework, designed to process large amounts of data in memory.


Run the following cell to install Spark 3.2.1 to your Colab instance (we'll use it later on):

In [ ]:
# Install Spark

import os
os.chdir("/content")
!test -f spark-3.2.1-bin-hadoop2.7.tgz || wget https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!test -d spark-3.2.1-bin-hadoop2.7 || tar -xf spark-3.2.1-bin-hadoop2.7.tgz

# Setup pyspark
!pip install findspark
import findspark
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"
findspark.init()

# Create new SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .master("local[*]") \
        .getOrCreate()

# Extracting, Transforming & Loading

## How do we extract the data

### Reading the data

There is a directory named `sample_data`, which contains housing market data from California from around 1990.
- to find out more about the data, see [its docs](https://developers.google.com/machine-learning/crash-course/california-housing-data-description)

We will load all the CSV that are named `california_housing*.csv` into Apache Spark session. But let's first take a prompt look into how does these data look:

In [ ]:
!head sample_data/california_housing*.csv

Two files matched our pattern and both have the same "schema", so we can read both at once into a Spark session using simple command (which also saves it into a variable):

In [ ]:
housing_market = spark.read.csv("sample_data/california_housing*.csv")

Well, the command didn't fail, so we might presume it have probably succeeded.
- to make sure it really had succeded, let's see the content of our new variable:

In [ ]:
housing_market

Now you can see that the variable `housing_market` holds something called `DataFrame`.
- but what is a `DataFrame` anyway?

You can imagine `DataFrame` like a simple table with rows and named columns with neat API you can call upon it very easily (as we'll see later).
- to learn more about DataFrames, [see official Spark Docs](https://spark.apache.org/docs/3.2.0/sql-programming-guide.html)

### Your first task

Now we know the `housing_market` holds a DataFrame inside.
- but we can also see that the column names are missing
  - *(from the previous console output you can see `_c0`, `_c1`, ... instead of the real column names)*

To fix this, we have to consult the [spark.read.csv API](https://spark.apache.org/docs/3.2.0/sql-data-sources-csv.html#data-source-option) first.
- make some effort, and try to find the option we could use to make sure the column names of the CSV are properly extracted

Found it? Try to modify the options within the function call and re-run the cell.
- *HINT: Wait a while after writing the `,` as Colab'll show you Spark API suggestions*

In [ ]:
# Change this call to create DF with respect of the column names from the CSV headers.
housing_market = spark.read.csv("sample_data/california_housing_test.csv")
housing_market

If you see real column names (`longitude`, `latitude`, ...) instead of (`_c0`, `_c1`, ...), you got it right.

Another way to show the DataFrame schema is simply calling:

In [ ]:
housing_market.printSchema()

Last thing you might find useful is to print a few lines of the underlying DataFrame:

In [ ]:
housing_market.show()

### Extraction complete

That's about all you need to know for the moment of how to extract the data.

Keep in mind that in the real world, it's actually way more difficult, since you have to tackle problems including, but not limited to:
- schema of one data source doesn't match the schema of another data source
- file formats are different between data sources (e.g. Parquet instead of CSV)
- you might have to crawl the data first from some API using pagination
- ...

## How do we transform the data

### What a transformation and why should I care?

In the previous chapter, we have extracted housing market data from CSV files into a single DataFrame under a variable called `housing_market`. In this chapter, we will focus on transforming the data into a different form. But what is the motivation of such transformation, anyway?

Well, as data grow big, it's harder to extract a particular value, since there is so much data, the computation would essentially take forever.

Because of this fact, it's common to process finite chunks of recent data and transform them into another form, so that we can:
- extract the desired value faster when needed
- prepare values for machine learning algorithms

### Transformation example

We will elaborate upon two transformation types:
- *group by* (aggregation)
- *user defined function* (or UDF for short)

#### Group By (aggregation)

Let's build an aggregation, which tells us, **how many housing blocks were built how many decades ago**. We will use the `housing_median_age` column. Note that one housing block should be one row in the DataFrame.

To achieve this, we need to complete these high-level steps:
- assign decade to each block (e.g. both `13` and `19` will be of 10th decade)
- group the blocks together by the defined decade
- count the groups that were formed for each decade

To define a new column, we can use the [`DF.withColumn` API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumn.html?highlight=withcolumn). To get a housing block median age in decades, we simply floor the `housing_median_age` to the closest ten and store it in a new column named `age_group`.

In [ ]:
from pyspark.sql.functions import col, floor

housing_market_with_age_group = housing_market.withColumn("age_group", floor(col("housing_median_age") / 10) * 10)

housing_market_with_age_group

Now we can group the DataFrame using our fresh column and perform count of each group:

In [ ]:
age_group_histogram = housing_market_with_age_group.groupBy("age_group").count()

age_group_histogram

And that's pretty much it! We might also want to enjoy looking at the resulting data.
- to do that, we should also **sort the DataFrame** so that even human can easily deduct conclusion:

In [ ]:
age_group_histogram = age_group_histogram.sort("age_group")

age_group_histogram.show()

We did this step-by-step to keep it simple, but it is actually more common to write the whole transformation into a single command like this:

In [ ]:
age_group_histogram = housing_market \
    .withColumn("age_group", floor(col("housing_median_age") / 10) * 10) \
    .groupBy(col("age_group")) \
    .count() \
    .sort("age_group")
age_group_histogram.show()

We can also easily convert our Spark DataFrame into a Pandas DataFrame for visualisation in Google Colab:

In [ ]:
import matplotlib.pyplot as plt

age_group_histogram.toPandas().plot(x="age_group")

You can see, that most housing blocks were built (using median age of all buildings in a block) somewhat between 30-40 years ago (because `age_group=30` is the floor of the interval). Maybe this metric could help someone from the real estate agency, but please note that we used only a simple `count`, yet there are many more actions you can perform upon [the `GroupedData` API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/grouping.html).

#### User Defined Function (UDF)

Sometimes the transformation we need to do, cannot be easily describe using the available API and we need to write our custom function instead.

Since all our columns are numbers, we should not need to write our own UDFs, but let's try to write it anyway. For example, we could write a function, that categorizes each block into one of:
- poor
  - `medianIncome < 1` and `population / totalRooms < 1`
- middle_class
  - `medianIncome between 1-3` and `population / totalRooms < 2`
- rich
  - `medianIncome > 3` and `population / totalRooms > 1`
- unknown
  - all other housing blocks




So we start by defining our function within pure Python:

In [ ]:
def categorize(median_income, population, total_rooms):
  # Since we focus on the basics, let's not dive into why we need to convert string to float
  median_income = float(median_income)
  population_over_rooms = float(population) / float(total_rooms)

  if median_income < 1 and population_over_rooms < 1:
    return "poor"
  elif median_income > 1 and median_income < 3 and population_over_rooms < 2:
    return "middle_class"
  elif median_income > 3 and population_over_rooms > 1:
    return "rich"
  else:
    return "unknown"

Now we can define our pure Python function as Spark UDF:

In [ ]:
from pyspark.sql.functions import udf

categorize_udf = udf(categorize)

Great, let's use it to categorize our loaded housing blocks and count the categories using `group by` we have already covered:

In [ ]:
housing_market \
  .groupBy(categorize_udf(col("median_income"), col("population"), col("total_rooms"))) \
  .count() \
  .sort("count") \
  .show()

And this is basically how we use UDF to extract algorithmically defined value.

Please note that we should always use Spark SQL API instead, where possible, because Spark can optimize the job and run efficiently. Instead, with UDF, the Python code is a black-box for Spark, so it cannot be optimized.

## Loading the data

*NOTE: It might be confusing, to talk about "loading" when we actually mean operation that saves data somewhere else. But that's definition of ETL, so we should respect this globally accepted naming convention.*

This last chapter will very briefly show-case, how to store the in-memory created DataFrame for later usage.

We will store it into a disposable Google Colab kernel instance, but normally, it should be stored into a data warehouse (Dremio, Hive, HBase, Impala, Kafka, HDFS, S3, ...).

The benefits of using a data warehouse include:
- to use the results later on using additional data pipelines or analytical tools
- to ensure data loss protection, durability and reliability
- to easily maintain data statistics such as their size, available history, etc.

Each data warehouse has its own documentation, from where you can find instructions of how to store your DataFrame from Spark.

For most of the warehouses, it's as simple, as calling `DataFrame.write.FORMAT(PATH)`. However in our case, we are good with writing only CSV file, so we use:

In [ ]:
age_group_histogram.write.csv("my_first_spark_output", header=True, mode="overwrite")

With this command, `my_first_spark_output` directory should be created:

In [ ]:
!ls -al my_first_spark_output

In [ ]:
!cat my_first_spark_output/*.csv

There you have it. Your own CSV aggregation result right from your browser.